In [2]:
from spread_models import dry_forest
import test_data_generator as tdg
import pandas as pd

In [3]:
# setup the defualt inputs
fuel_params_dict = {
    'FHS_s': 3,
    'FHS_ns': 3,
    'FL_s': 10,
    'FL_ns': 3.5,
    'FL_el': 2,
    'FL_b': 2,
    'FL_o': 4.5,
    # 'FL_total': 12,
    'Fk_s': 0.3,
    'Fk_ns': 0.3,
    'Fk_el': 0.3,
    'Fk_b': 0.3,
    # 'Fk_o': 0.3,
    'Hk_ns': 0.3,
    # 'Cov_o': 40,
    'H_ns': 20,
    'H_el': 2,
    'H_o': 20,
    'WRF_For': 3,
}

fuel_params_df = pd.DataFrame(fuel_params_dict, index=[0])

datetime_param_dict = {
    'datetime': ('2022-01-01', '2022-09-01', 90, 6),
}

num_param_dict = {
    'WindMagKmh_SFC': (10,50,20),
    'RH_SFC': (10,100,50),
    'T_SFC': (10,40,15),
    # 'precipitation': (0,100,50),
    # 'time_since_rain': (0,48,24),
    'time_since_fire': (0,15,7.5),
    'DF_SFC': (2,10,4),
}

class_param_dict = {}

df = tdg.generate_test_data(datetime_param_dict,num_param_dict,class_param_dict)
df['months'] = df.datetime.dt.month
df['hours'] = df.datetime.dt.hour


In [4]:
# generate outputs
output_dict =dry_forest.calculate(df.to_xarray(),fuel_params_df.iloc[0])

for param, series in output_dict.items():
    df[param] = series

print(df.head())
print(df.shape)

    datetime  WindMagKmh_SFC  RH_SFC  T_SFC  time_since_fire  DF_SFC  months  \
0 2022-01-01            10.0    10.0   10.0              0.0     2.0       1   
1 2022-01-01            10.0    10.0   10.0              0.0     6.0       1   
2 2022-01-01            10.0    10.0   10.0              0.0    10.0       1   
3 2022-01-01            10.0    10.0   10.0              7.5     2.0       1   
4 2022-01-01            10.0    10.0   10.0              7.5     6.0       1   

   hours  dead_fuel_moisture  rate_of_spread  flame_height   intensity  \
0      0               4.577       56.648558      1.375288    0.000000   
1      0               4.577       56.648558      1.375288    0.000000   
2      0               4.577       56.648558      1.375288    0.000000   
3      0               4.577       84.660937      1.838878  121.306613   
4      0               4.577      213.271674      3.586383  916.760387   

   spotting_distance  rating_1  index_1  
0          50.000000       0.0  

In [17]:
# sensitivity to surface fuel load 0 - 2- step of 2
fuel_params_dict_list = []
for fl_s in range(10,13,2):
    d = dict(fuel_params_dict)
    # d = d.update(fuel_params_dict)
    d['FL_s'] = fl_s
    fuel_params_dict_list.append(d)

fuel_params_df = pd.DataFrame(fuel_params_dict_list)

output_dict_list = []
for i, row in fuel_params_df.iterrows():
    od = dry_forest.calculate(df.to_xarray(),row)
    output_dict_list.append(od)

print(output_dict_list)


[{'dead_fuel_moisture': array([ 4.577,  4.577,  4.577, ..., 20.39 , 20.39 , 20.39 ]), 'rate_of_spread': array([56.64855799, 56.64855799, 56.64855799, ...,  7.58830668,
       35.54100631, 77.28318018]), 'flame_height': array([1.37528773, 1.37528773, 1.37528773, ..., 0.32150133, 0.98178323,
       1.72156517]), 'intensity': <xarray.DataArray (index: 3888)>
array([  0.        ,   0.        ,   0.        , ...,  10.46809144,
       147.08676914, 612.0340048 ])
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 ... 3881 3882 3883 3884 3885 3886 3887, 'spotting_distance': <xarray.DataArray 'WindMagKmh_SFC' (index: 3888)>
array([50., 50., 50., ..., 50., 50., 50.])
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 ... 3881 3882 3883 3884 3885 3886 3887, 'rating_1': array([0., 0., 0., ..., 0., 0., 0.]), 'index_1': array([ 0.,  0.,  0., ...,  0.,  6., 10.])}, {'dead_fuel_moisture': array([ 4.577,  4.577,  4.577, ..., 20.39 , 20.39 , 20.39 ]), 'rate_of_spread': array([56.64855799, 56.6485